In [ ]:
#ANANT MEHTA 102003556 COE21

#Question 1
(Based on Step-by-Step Implementation of Ridge Regression using Gradient
Descent Optimization)
Generate a dataset with atleast seven highly correlated columns and a target variable.
Implement Ridge Regression using Gradient Descent Optimization. Take different
values of learning rate (such as 0.0001,0.001,0.01,0.1,1,10) and regularization
parameter (10-15,10-10,10-5
,10- 3
,0,1,10,20). Choose the best parameters for which ridge

regression cost function is minimum and R2_score is maximum.

In [83]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import math

In [84]:
#Generating highly correlated dataset
n = 10
r = 0.8
x1 = np.random.rand(1,5)
x2 = np.random.rand(1,5)
y1 = r*x2+ math.sqrt(1-r*r)*x1 
y2 = r*y1+ math.sqrt(1-r*r)*x2
y3 = r*y2+ math.sqrt(1-r*r)*y1
y4 = r*y3+ math.sqrt(1-r*r)*y2
y5 = r*y4+ math.sqrt(1-r*r)*y3
y = np.random.rand(1,5)
x1=x1.T
x2=x2.T
y1=y1.T
y2=y2.T
y3=y3.T
y4=y4.T
y5=y5.T
y=y.T
df1=np.concatenate((x1,x2),axis=1)
df2=np.concatenate((y1,y2),axis=1)
df3=np.concatenate((y3,y4),axis=1)
df4=np.concatenate((y5,y),axis=1)
df5=np.concatenate((df1,df2),axis=1)
df6=np.concatenate((df3,df4),axis=1)
df=np.concatenate((df5,df6),axis=1)
print(df.shape)

(5, 8)


In [85]:
df = pd.DataFrame(df,columns=['x1', 'x2', 'y1', 'y2', 'y3','y4','y5','y'])

In [86]:
df#Dataset

,x1,x2,y1,y2,y3,y4,y5,y
0,0.345561,0.204452,0.370898,0.419390,0.558051,0.698075,0.893290,0.558690
1,0.396767,0.878117,0.940554,1.279314,1.587784,2.037815,2.582923,0.140387
2,0.538817,0.027388,0.345200,0.292593,0.441194,0.528511,0.687525,0.198101
3,0.419195,0.670468,0.787891,1.032593,1.298809,1.658603,2.106168,0.800745
4,0.685220,0.417305,0.744976,0.846363,1.124076,1.407079,1.800109,0.968262


In [87]:
X = df.iloc[:, :-1].values
y = df.iloc[::, 7].values  
print(X)
print(y)

[[0.34556073 0.20445225 0.37089824 0.41938994 0.55805089 0.69807468
  0.89329028]
 [0.39676747 0.87811744 0.94055443 1.27931401 1.58778387 2.0378155
  2.58292272]
 [0.53881673 0.02738759 0.34520011 0.29259265 0.44119419 0.52851094
  0.68752526]
 [0.41919451 0.67046751 0.78789072 1.03259308 1.29880889 1.65860296
  2.10616771]
 [0.6852195  0.4173048  0.74497554 0.84636332 1.12407598 1.40707877
  1.8001086 ]]
[0.55868983 0.14038694 0.19810149 0.80074457 0.96826158]


In [88]:
def compute_cost(X, y, w, b, lambda_= 1):
    m, n = X.shape
    total_cost=0.0
    for i in range(m):
        z = np.dot(X[i],w) +b
        f_w = z
        total_cost = total_cost + (-y[i]*np.log(f_w)) -(1-y[i])*np.log(1-f_w)
    total_cost = total_cost/m
    return total_cost

In [89]:
def compute_gradient(X, y, w, b, lambda_=None): 
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.
    for i in range(m):
        f_wb_i = np.dot(X[i],w) + b          #(n,)(n,)=scalar
        err_i  = f_wb_i  - y[i]                       #scalar
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]      #scalar
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m                                   #(n,)
    dj_db = dj_db/m  
    return dj_db, dj_dw

In [90]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    m = len(X)
    J_history = []
    w_history = [] 
    for i in range(num_iters):
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)  
        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
        # Save cost J at each iteration
        if i<100000:      
            cost =  cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)  
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
    return w_in, b_in, J_history, w_history 

In [91]:
def compute_cost_reg(X, y, w, b, lambda_):
    m, n = X.shape
    cost_without_reg = compute_cost(X, y, w, b)    
    reg_cost = 0.
    ### START CODE HERE ###
    for j in range(n):
        reg_cost_j = w[j]**2
        reg_cost = reg_cost + reg_cost_j      
    # total cost
    total_cost = cost_without_reg + (lambda_/(2 * m)) * reg_cost
    return total_cost

In [92]:
def compute_gradient_reg(X, y, w, b, lambda_): 
    m, n = X.shape  
    dj_db, dj_dw = compute_gradient(X, y, w, b, lambda_)   
    for j in range(n): 
        dj_dw_j_reg = (lambda_ *w[j])/m
          # Added the regularization term  
        dj_dw[j] = dj_dw[j] + dj_dw_j_reg          
    return dj_db, dj_dw

In [93]:
np.random.seed(1)
initial_w = np.random.rand(X.shape[1])-0.5
initial_b = 1.
lambda_ = 0.0001;                                          
iterations = 1000
alpha = 0.01
w,b, J_history,_ = gradient_descent(X, y, initial_w, initial_b, 
                                    compute_cost_reg, compute_gradient_reg, 
                                    alpha, iterations, lambda_)
print(w,b)

Iteration    0: Cost      nan   
Iteration  100: Cost     0.81   
Iteration  200: Cost     0.78   
Iteration  300: Cost     0.76   
Iteration  400: Cost     0.74   
Iteration  500: Cost     0.74   
Iteration  600: Cost     0.73   
Iteration  700: Cost     0.72   
Iteration  800: Cost     0.72   
Iteration  900: Cost     0.72   
Iteration  999: Cost     0.71   
[-0.01500466  0.35245172 -0.35327458 -0.00111627 -0.10805752 -0.09358014
  0.08459231] 0.7226838100856741


C:\Users\Anant Mehta\AppData\Local\Temp\ipykernel_17904\1402940483.py:7: RuntimeWarning: invalid value encountered in log
  total_cost = total_cost + (-y[i]*np.log(f_w)) -(1-y[i])*np.log(1-f_w)


In [94]:
np.random.seed(1)
initial_w = np.random.rand(X.shape[1])-0.5
initial_b = 1.
lambda_ = 0.0001;                                          
iterations = 1000
alpha = 0.01
w,b, J_history,_ = gradient_descent(X, y, initial_w, initial_b, 
                                    compute_cost_reg, compute_gradient_reg, 
                                    alpha, iterations, lambda_)
print(w,b)

Iteration    0: Cost      nan   
Iteration  100: Cost     0.81   
Iteration  200: Cost     0.78   
Iteration  300: Cost     0.76   
Iteration  400: Cost     0.74   
Iteration  500: Cost     0.74   
Iteration  600: Cost     0.73   
Iteration  700: Cost     0.72   
Iteration  800: Cost     0.72   
Iteration  900: Cost     0.72   
Iteration  999: Cost     0.71   
[-0.01500466  0.35245172 -0.35327458 -0.00111627 -0.10805752 -0.09358014
  0.08459231] 0.7226838100856741


C:\Users\Anant Mehta\AppData\Local\Temp\ipykernel_17904\1402940483.py:7: RuntimeWarning: invalid value encountered in log
  total_cost = total_cost + (-y[i]*np.log(f_w)) -(1-y[i])*np.log(1-f_w)


In [95]:
np.random.seed(1)
initial_w = np.random.rand(X.shape[1])-0.5
initial_b = 1.
lambda_ = 10** (-5);                                          
iterations = 1000  
alpha = 0.1
w,b, J_history,_ = gradient_descent(X, y, initial_w, initial_b, 
                                    compute_cost_reg, compute_gradient_reg, 
                                    alpha, iterations, lambda_)
print(w,b)

Iteration    0: Cost     0.90   
Iteration  100: Cost     0.71   
Iteration  200: Cost     0.70   
Iteration  300: Cost     0.69   
Iteration  400: Cost     0.68   
Iteration  500: Cost     0.67   
Iteration  600: Cost     0.66   
Iteration  700: Cost     0.66   
Iteration  800: Cost     0.66   
Iteration  900: Cost     0.65   
Iteration  999: Cost     0.65   
[ 0.76557428  0.01540285 -0.15456632 -0.04437898 -0.02344273 -0.05184594
  0.16874855] 0.114020907457255


In [96]:
np.random.seed(1)
initial_w = np.random.rand(X.shape[1])-0.5
initial_b = 1.
lambda_ = 10** (-3);                                          
iterations = 1000
alpha = 0.1
w,b, J_history,_ = gradient_descent(X, y, initial_w, initial_b, 
                                    compute_cost_reg, compute_gradient_reg, 
                                    alpha, iterations, lambda_)
print(w,b)
#Like this alpha and lamda combinations are used and the one with least cost is selected

Iteration    0: Cost     0.90   
Iteration  100: Cost     0.71   
Iteration  200: Cost     0.70   
Iteration  300: Cost     0.69   
Iteration  400: Cost     0.68   
Iteration  500: Cost     0.67   
Iteration  600: Cost     0.66   
Iteration  700: Cost     0.66   
Iteration  800: Cost     0.66   
Iteration  900: Cost     0.65   
Iteration  999: Cost     0.65   
[ 0.75934193  0.01133165 -0.14928509 -0.04396973 -0.02200125 -0.05032047
  0.16643649] 0.1154220583502929


#Question 2
Load the Hitters dataset from the following link
https://drive.google.com/file/d/1qzCKF6JKKMB0p7ul_lLy8tdmRk3vE_bG/view?usp=sharing
(a) Pre-process the data (null values, noise, categorical to numerical encoding)
(b) Separate input and output features and perform scaling
(c) Fit a Linear, Ridge (use regularization parameter as 0.5748), and LASSO (use
regularization parameter as 0.5748) regression function on the dataset.
(d) Evaluate the performance of each trained model on test set. Which model performs
the best and Why?

In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('Hitters.csv')

In [5]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [9]:
df.isnull().sum()

AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary       59
NewLeague     0
dtype: int64

In [10]:
#filling the null values
df=df.fillna(df.mean())

C:\Users\Anant Mehta\AppData\Local\Temp\ipykernel_17904\2389315911.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df=df.fillna(df.mean())


In [11]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,535.925882,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.000000,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.000000,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.000000,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.500000,N


In [12]:
#label encoding
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
df['League']= label_encoder.fit_transform(df['League']) 
df['Division']= label_encoder.fit_transform(df['Division']) 
df['NewLeague']= label_encoder.fit_transform(df['NewLeague']) 
df.head(20)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,0,0,446,33,20,535.925882,0
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,1,1,632,43,10,475.000000,1
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,0,1,880,82,14,480.000000,0
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,1,0,200,11,3,500.000000,1
4,321,87,10,39,42,30,2,396,101,12,48,46,33,1,0,805,40,4,91.500000,1
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,0,1,282,421,25,750.000000,0
6,185,37,1,23,8,21,2,214,42,1,30,9,24,1,0,76,127,7,70.000000,0
7,298,73,0,24,24,7,3,509,108,0,41,37,12,0,1,121,283,9,100.000000,0
8,323,81,6,26,32,8,2,341,86,6,32,34,8,1,1,143,290,19,75.000000,1
9,401,92,17,49,66,65,13,5206,1332,253,784,890,866,0,0,0,0,0,1100.000000,0


In [13]:
X=df.iloc[:,:-1].values
Y=df.iloc[:,-1].values

In [15]:
X,Y#numpy arrays

(array([[ 293.        ,   66.        ,    1.        , ...,   33.        ,
           20.        ,  535.92588213],
        [ 315.        ,   81.        ,    7.        , ...,   43.        ,
           10.        ,  475.        ],
        [ 479.        ,  130.        ,   18.        , ...,   82.        ,
           14.        ,  480.        ],
        ...,
        [ 475.        ,  126.        ,    3.        , ...,  113.        ,
            7.        ,  385.        ],
        [ 573.        ,  144.        ,    9.        , ...,  131.        ,
           12.        ,  960.        ],
        [ 631.        ,  170.        ,    9.        , ...,    4.        ,
            3.        , 1000.        ]]),
 array([0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
        0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
        1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
     

In [18]:
X.shape,Y.shape

((322, 19), (322,))

In [19]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
X_train, X_test, y_train, y_test= train_test_split(X,Y, test_size=0.3, random_state=100)

In [21]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train_transformed=sc.fit_transform(X_train)
X_test_transformed=sc.transform(X_test)

In [22]:
X_train_transformed

array([[ 1.51524373,  1.33860078,  1.82188402, ...,  1.45996728,
         1.01322546, -0.77337458],
       [ 0.84995114,  0.9259886 , -1.25425423, ...,  2.53872418,
         1.01322546,  3.33617211],
       [ 1.33913687,  1.4906158 ,  3.30298762, ..., -0.68266713,
        -0.84023575,  1.67700168],
       ...,
       [-0.29148222, -0.11640007, -0.79853004, ...,  0.75319551,
         0.08649486, -0.59623895],
       [ 0.45208008,  0.03561495, -0.34280586, ...,  2.19649785,
        -0.37687045, -0.99774638],
       [-0.39584184, -0.44214653, -0.57066795, ...,  1.32605263,
         1.63104587, -1.06860064]])

In [36]:
def performance(x_test,y_pred,y_test,model):
    from sklearn.metrics import mean_squared_error
    score=model.score(X_test,y_test)
    mse=mean_squared_error(y_test, y_pred)
    return mse

In [37]:
#Linear Regression
from sklearn.metrics import accuracy_score
from sklearn import linear_model
model1 = linear_model.LinearRegression()
model1.fit(X_train_transformed, y_train)
Y_pred=model1.predict(X_test_transformed)
print("Performance of Linear Regression")
mse=performance(X_test_transformed,Y_pred,y_test,model1)
print(f"MSE is {mse} ")

#Ridge Regression
from sklearn.linear_model import Ridge
model2 = Ridge(alpha=0.5748)
model2.fit(X_train_transformed, y_train)
Y_pred=model2.predict(X_test_transformed)
print("Performance of Ridge Regression")
mse=performance(X_test_transformed,Y_pred,y_test,model2)
print(f"MSE is {mse} ")

#Lasso Regression
model3 = linear_model.Lasso(alpha=0.5748)
model3.fit(X_train_transformed, y_train)
Y_pred=model3.predict(X_test_transformed)
print("Performance of Lasso Regression")
mse=performance(X_test_transformed,Y_pred,y_test,model3)
print(f"MSE is {mse} ")

Performance of Linear Regression
MSE is 0.10147339015722112 
Performance of Ridge Regression
MSE is 0.10063768133020616 
Performance of Lasso Regression
MSE is 0.25174432989690726 


#So , it can be said that least error is on fitting the data with Ridge Regression
As there is less overfitting due to regularization

#Question 3
Explore Ridge Cross Validation (RidgeCV) and Lasso Cross Validation (LassoCV)
function of Python. Implement both on Boston House Prediction Dataset (load_boston
dataset from sklearn.datasets).

In [39]:
from sklearn import datasets
boston=datasets.load_boston()
X=boston.data
Y=boston.target
print(X.shape)
print(Y.shape)

(506, 13)
(506,)


C:\Users\Anant Mehta\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dat

In [40]:
X

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [41]:
df=pd.DataFrame(X)
#print(boston.feature_names)
df.columns=boston.feature_names
print(df.describe())

             CRIM          ZN       INDUS        CHAS         NOX          RM  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695    6.284634   
std      8.601545   23.322453    6.860353    0.253994    0.115878    0.702617   
min      0.006320    0.000000    0.460000    0.000000    0.385000    3.561000   
25%      0.082045    0.000000    5.190000    0.000000    0.449000    5.885500   
50%      0.256510    0.000000    9.690000    0.000000    0.538000    6.208500   
75%      3.677083   12.500000   18.100000    0.000000    0.624000    6.623500   
max     88.976200  100.000000   27.740000    1.000000    0.871000    8.780000   

              AGE         DIS         RAD         TAX     PTRATIO           B  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean    68.574901    3.795043    9.549407  408.237154   18.455534  356.674032   
std     28.148861    2.1057

In [69]:
#Exploring RidgeCV and LassoCV and fitting data for CV=3
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
for cv in range(2,10):
    print(f"when cv is set to {cv}")
    for Model in [LassoCV, RidgeCV]:
        mod=Model(alphas=[1e-3, 1e-2, 1e-1, 1],cv=cv).fit(X,Y)
        score=mod.score(X,Y)
        print(f"Score for {Model} is {score}")

when cv is set to 2
Score for <class 'sklearn.linear_model._coordinate_descent.LassoCV'> is 0.6825842212709925
Score for <class 'sklearn.linear_model._ridge.RidgeCV'> is 0.7388703133867616
when cv is set to 3
Score for <class 'sklearn.linear_model._coordinate_descent.LassoCV'> is 0.7269834862602695
Score for <class 'sklearn.linear_model._ridge.RidgeCV'> is 0.7388703133867616
when cv is set to 4
Score for <class 'sklearn.linear_model._coordinate_descent.LassoCV'> is 0.6825842212709925
Score for <class 'sklearn.linear_model._ridge.RidgeCV'> is 0.7388703133867616
when cv is set to 5
Score for <class 'sklearn.linear_model._coordinate_descent.LassoCV'> is 0.7269834862602695
Score for <class 'sklearn.linear_model._ridge.RidgeCV'> is 0.7388703133867616
when cv is set to 6
Score for <class 'sklearn.linear_model._coordinate_descent.LassoCV'> is 0.6825842212709925
Score for <class 'sklearn.linear_model._ridge.RidgeCV'> is 0.7388703133867616
when cv is set to 7
Score for <class 'sklearn.linear_mo

Explored RidgeCV and LassoCV with possible values of CV and alphas

Score for <class 'sklearn.linear_model._coordinate_descent.LassoCV'> is 0.7402097746210394
Score for <class 'sklearn.linear_model._ridge.RidgeCV'> is 0.7388703133867616
Here Lasso performed better than Ridge